# Introduction

The SnP500 program has the following components:
1. Ticker and expiry extraction - Make a list of NYSE weekly tickers of interest
2. Contract data extraction - Probablility of Profit from Standard Deviation, Strike Price, Underlying Value
3. Determine margins and Return on Margin
4. Assemble target dataframe
5. Place orders

## Ticker and Expiry extraction
The following program called <i>snp_weeklies</i>:
1. Generates a list of NYSE weekly options from CBOE's spreadsheet
2. Extracts only S&P500 scrips from it

The file contents <i> snp_weeklies.py</i> can be seen here...

## Contract Data Extraction

This program is split into the following parts:
1. Assemble the weeklies dataframe
2. Connect to IBKR
3. Extract target scrips underlying price and standard deviation
4. Extract all strike prices for the expiries
4. Filter dataframe for only relevant strike prices beyond standard deviation threshold
5. Extract the contract price for the filtered dataframe

### 1) Assemble the weeklies dataframe
The script is imported. Weeklies df is generated.

In [ ]:
import snp_weeklies

# Get the weeklies dataframe
df = snp_weeklies.main()

# limit the DTE to a reasonable no of days
df1 = df.loc[(df.DTE > 30) & (df.DTE < 60), :]

# Small dataframe
df_small = df1.head().copy()
df_small

### 2) Connect to IBKR   
<p>...Ensure that either TWS or IBG has been started in _Paper_ mode.</p>
<p>...In _File -> Global Configuration -> API -> Settings_ the Socket number typically for TWS is: 7497 | IBG is: 4002</p>
<p>...Read-Only API is unchecked</p>
<p>...Trusted IPs is 127.0.0.1</p>

To set up a connection with IBKR a connector class has been made. The file contents of _connector.py_ is in the cell below.

Run the following cells to:<br\>
a) Instantiate <i>ib_insync</i><br\>
b) Instantiate a connection<br\>

In [ ]:
from ib_insync import *
util.startLoop()  # to keep Jupyter live

# Instantiate from main IB class
ib = IB()

from connector import Connector

platform = 2       # (<1> for TWS | <2> for IBG)
live_or_paper = 2  # (<1> for Live | <2> for Paper)
y = 1              # (<1> for USA  | <2> for NSE)

connection = Connector(platform = platform, live_or_paper = live_or_paper, y = y)

ip=connection.ip
socket=connection.socket
client = connection.client

# Connect to IB
ib.connect(host=ip, port=socket, clientId=client)

# Set the exchange
exchange = 'SMART'

# Set currency and primaryExchange based on connector
if connection.y == 1:
    currency  = 'USD'
    primaryExchange = 'NASDAQ'
else:
    currency = 'INR'
    primaryExchange = 'NSE'

### 3) Extract underlying price and standard deviation

In [ ]:
import pandas as pd
import numpy as np

def get_price(symbol, exchange=exchange, currency=currency, primaryExchange=primaryExchange):
    '''Gets the Last / Market Price of the Stock
    Args: 
       (symbol)          = string: name of equity
       
       [exchange]        = string: [default: 'SMART']
       [currency]        = string: [default: 'USD']
       [primaryExchange] = string: for scrips like INTC [default: 'NASDAQ']
       
    Returns:
           marketPrice() = float
           
    '''
    contract = Stock(symbol, exchange=exchange, currency=currency, primaryExchange=primaryExchange)
    ib.qualifyContracts(contract) # qualify the contract
    
    [ticker] = ib.reqTickers(contract)
    price = ticker.marketPrice()
    
    return price

def get_stdev(symbol, DTE, exchange=exchange, currency=currency, primaryExchange=primaryExchange):
    '''Gets the Standard Deviation of the Stock
    Args: 
       (symbol)          = string: name of equity
       (DTE)             = int: days-to-expiry
       
       [exchange]        = string: [default: 'SMART']
       [currency]        = string: [default: 'USD']
       [primaryExchange] = string: for scrips like INTC [default: 'NASDAQ']
       
    Returns:
       standard deviation = float
    
    '''
    contract = Stock(symbol, exchange=exchange, currency=currency, primaryExchange=primaryExchange)
    
    sd_days = str(DTE)+' D'  # days for standard deviation
    bars = ib.reqHistoricalData(contract=contract, endDateTime='', durationStr=sd_days, 
                                barSizeSetting='1 day',  whatToShow='Trades', useRTH=True)
    sd = np.std([b.close for b in bars], ddof=1)
    
    return sd

# Get the unique list of symbols to generate a price dataframe
underlying_df = pd.DataFrame(df_small['Ticker'].unique().tolist(), columns=['Ticker'])

# Make the price dataframe
underlying_df['Underlying'] = underlying_df.Ticker.apply(get_price)

# Make the Standard Deviation dataframe
df_small.loc[:, 'Stdev'] = df_small[['Ticker', 'DTE']].apply(lambda x: get_stdev(*x), axis=1)

# Merge the underlying and stdev dataframes
df_small = df_small.merge(underlying_df, how='left', left_on='Ticker', right_on='Ticker')

In [ ]:
df_small

In [ ]:
scrip = Option(symbol='ABBV', lastTradeDateOrContractMonth='20180810', exchange='SMART')

In [ ]:
ib.qualifyContracts(scrip)

In [ ]:
set(cd.scrip.strike for cd in ib.reqContractDetails(scrip))

In [ ]:
help(Option)

### 3) Extract strike prices for the expiries

In [ ]:
def get_strikes(symbol, expiry, exchange=exchange, currency=currency, primaryExchange=primaryExchange):
    '''Gets the strike prices for the various expiries of a contract
    Args: 
       (symbol)          = string: name of equity
       (expiry)          = string: expiry in yyyymmdd format
       
       [exchange]        = string: [default: 'SMART']
       [currency]        = string: [default: 'USD']
       [primaryExchange] = string: for scrips like INTC [default: 'NASDAQ']
       
    Returns:
       strikes = tuple of floats
    
    '''
    contract = Option(symbol, expiry, exchange=exchange, currency=currency, primaryExchange=primaryExchange)
    
    strikes = set(cd.contract.strike for cd in ib.reqContractDetails(contract))
    
    return strikes

In [ ]:
get_strikes(symbol='AAPL', expiry='20180810')

## Prepare to Order
6. Check out the margin requirements for the trade
7. Assemble scrips + strike, sorted and weighted PoP and RoM

## Order Management
8. Check liquidation value limits
9. Place orders with continuous liquidation value limit check

In [ ]:
contract = Option('AAPL', '20180622', exchange='SMART')
set(cd.contract.strike for cd in ib.reqContractDetails(contract))

### 3) Extract target scrips info

In [ ]:
contract = Option('AAPL', '20180622', exchange='SMART')
set(cd.contract.strike for cd in ib.reqContractDetails(contract))``

In [ ]:
exch = 'SMART'
curr = 'USD'
scrip = 'TSLA'
stock = Stock(symbol=scrip, currency=curr, exchange=exch)
vol = ib.reqHistoricalData(contract=stock, endDateTime='', durationStr='33 D', 
                                barSizeSetting='1 DAY', whatToShow='OPTION_IMPLIED_VOLATILITY', 
                                useRTH=False)

In [ ]:
np.mean([v.close for v in vol])

In [ ]:
import numpy as np
import math
bars = ib.reqHistoricalData(contract=stock, endDateTime='', durationStr='33 D', 
                                barSizeSetting='1 DAY', whatToShow='Trades', 
                                useRTH=False)
daily_sd = np.std([b.close for b in bars], ddof=1)
daily_sd

3) Get StdDev for the expiries

In [ ]:
df2 = df1.head()
df2

def get_stdev(scrip, DTE):
    '''
    Gets the standard deviation of the stock.
    Min DTE is 45 days
    
    Args: scrip = name of scrip (string)
           DTE = Days to Expiry (int)
           
    Returns: Std Dev of scrip (double?) '''
    
    exch = 'SMART'
    curr = 'USD'
    sd_days = str(DTE)+' D'
    
    stock = Stock(scrip, exch, curr)
    bars = ib.reqHistoricalData(stock, '', sd_days, '1 day', 'Trades', True )
    
    return np.std([b.close for b in bars], ddof=1)

In [ ]:
# Put the parameters
scrip = 'AAPL'
exch = 'SMART'
curr = 'USD'

# Qualify the contract
contract = Stock(scrip, exch, curr)
ib.qualifyContracts (contract)

cds = ib.reqContractDetails(contract)
cds

In [ ]:
cds = ib.reqContractDetails(Option('AAPL', '20180706', exchange='SMART', strike=152.5, right='P'))
options = [cd.contract for cd in cds]
tickers = [t for i in range(0, len(options), 100) for t in ib.reqTickers(*options[i:i + 100])]
tickers

In [ ]:
ib.reqContractDetails(Option('AAPL', '20180706', exchange='SMART', strike=152.5, right='P'))

In [ ]:
symbol = df2.iloc[0].Ticker
exp_date = df2.iloc[0].Expiry

In [ ]:
# contract = Option[symbol, exp_date, 240, 'C', 'SMART']
spx = Index('SPX', 'CBOE')
ib.qualifyContracts(spx)

In [ ]:
[ticker] = ib.reqTickers(spx)
ticker

In [ ]:
spxValue= ticker.marketPrice
spxValue

In [ ]:
import ib_insync
print(ib_insync.__all__)

In [ ]:
ib.isConnected() 

In [ ]:
# Disconnect
ib.disconnect()

In [ ]:
contract = Stock('BULLSHIT', exchange=exchange, currency=currency, primaryExchange=primaryExchange)

In [ ]:
try:
    ib.qualifyContracts(contract)
except Exception:
    pass